## Import Yahoo Finance API

## Setup
- Really important to get things working properly [GitHub Pull](https://github.com/ranaroussi/yfinance/pull/480/files). Affects base.py  
- General note, in order to determine the location of a python library (intalled by pip), run the command pip(3) show 'library name'
- In addition to the pull made above I actually followed the GitHub Pull [Here](https://github.com/ranaroussi/yfinance/pull/174). The author of this pull created their own branch [Here](https://github.com/ohenley/yfinance/tree/patch-1), I copied the file [base.py](https://github.com/ohenley/yfinance/blob/patch-1/yfinance/base.py) from this branch and replaced the base.py file in the initial install. So far everything appears to work  
## To Do  
- Check the yahoo_fin section of this [article](https://algotrading101.com/learn/yfinance-guide/) it tells how to download all of the markers (done)


## Gen Notes
- Data is up to date at least within previous close

## Things I want to Record
- [NCAV](https://cabotwealth.com/daily/value-investing/bargain-stocks-net-current-asset-value/)
- PE Ratio Graham liked to see it be below 9 (but positive obviously)
- [Current Assets](https://www.investopedia.com/terms/c/currentassets.asp) :All Current assets represent all the assets of a company that are expected to be conveniently sold, consumed, used, or exhausted through standard business operations with one year. Includes cash, cash equivalents, accounts receivable, stock inventory, marketable securities, pre-paid liabilities, and other liquid assets (Available via call to Balance Sheets as 'currentAssets')
- [Total Liabilities](https://www.investopedia.com/terms/t/total-liabilities.asp) : Total liabilities are the combined debts and obligations that an individual or company owes to outside parties. Everything the company owns is classified as an asset and all amounts the company owes for future obligations are recorded as liabilities. (Available via call to Balance Sheets as 'totalLiabilities')
- [Current Liabilities](https://www.investopedia.com/terms/c/currentliabilities.asp) :This one is more complex as the call to balance sheet odes not include it as a column. My best guess would be to call 'Balance Sheets and then add 'accountsPayable','CurrentLongTermDebt','otherCurrentLiabilities'

In [2]:
import yfinance as yf
import pandas as pd
import numpy as np


## Load Balance Sheet Data and Definitions of Included Data
- Treasury Shares (shares bought back by the company)
- Total Stockholder Equity
- Common Stock
- Other StockHolder Equity

# To Do
- Using the above values I should be able to calculate the amount of preferred stock outstanding for a given company

In [9]:
msft = yf.Ticker("SAVE")

# show balance sheet
a = msft.balance_sheet

print(a)
print(a.columns)

dfa = pd.DataFrame(a) # I believe it is actaully already a pandas dataframe
print(dfa.columns)



                             2019-12-31    2018-12-31    2017-12-31  \
Capital Surplus            3.793800e+08  3.712250e+08  3.601530e+08   
Total Liab                 4.782080e+09  3.236953e+09  2.383226e+09   
Total Stockholder Equity   2.261332e+09  1.928504e+09  1.762574e+09   
Deferred Long Term Liab    3.616030e+08  2.490100e+08  9.991500e+07   
Other Current Liab         3.154080e+08  2.919810e+08  2.637110e+08   
Total Assets               7.043412e+09  5.165457e+09  4.145800e+09   
Common Stock               7.000000e+03  7.000000e+03  7.000000e+03   
Other Current Assets       1.052060e+08  1.069010e+08  1.756150e+08   
Retained Earnings          1.955187e+09  1.625481e+09  1.469732e+09   
Other Liab                 4.915690e+08  3.776440e+08  3.313950e+08   
Treasury Stock            -7.324200e+07 -6.820900e+07 -6.731800e+07   
Other Assets               7.581120e+08  7.039790e+08  6.252220e+08   
Cash                       9.789570e+08  1.004733e+09  8.008490e+08   
Total 

## Items In Total Current Liabilities from Above List
- Accounts Payable 
- Other Current Liabilities
- Short Long Term Debt (long term debt due in next 12 monthes)
- 

In [8]:
ts = a.columns[0] #Selects the first data for the balance data
g = a[ts]
print('Total Current Assets:',g['Total Current Assets'])
print('Total Current Liabs:',g['Total Current Liabilities'])
print('Total Liab',g['Total Liab'])

debtLoad = g['Total Liab']/g['Total Current Assets']
CurrentRat = g['Total Current Assets']/g['Total Current Liabilities']
quickRat = (g['Total Current Assets']-g['Inventory'])/g['Total Current Liabilities']
NCAV = g['Total Current Assets']-g['Total Current Liabilities']
print('ratio of Total Liabilities to Current Assets(debt load):',debtLoad)
print('Current Ratio (Current Assets/Current Liabilities):',CurrentRat)  #We want this to be at around 1.5 or greater
print('Quick Ratio ((CurrentAssets-Inventory)/Current Liabilities):',quickRat)
print('Net Current Assets Values',NCAV)
      
# For some reason I cannot figure out what all on the balance sheet goes into the calculation of the 
# Total Current Liabilities
f = g['Total Current Liabilities']-g['Other Current Liab']-g['Accounts Payable']-g['Short Long Term Debt']
print(f/np.power(10,10),'e10')

print(g)


Total Current Assets: 8206000000.0
Total Current Liabs: 18311000000.0
Total Liab 60113000000.0
ratio of Total Liabilities to Current Assets(debt load): 7.325493541311236
Current Ratio (Current Assets/Current Liabilities): 0.44814592321555347
Quick Ratio ((CurrentAssets-Inventory)/Current Liabilities): 0.34705914477636396
Net Current Assets Values -10105000000.0
0.5499 e10
Intangible Assets                   2.084000e+09
Capital Surplus                     3.945000e+09
Total Liab                          6.011300e+10
Total Stockholder Equity           -1.180000e+08
Other Current Liab                  8.001000e+09
Total Assets                        5.999500e+10
Common Stock                        4.000000e+06
Other Current Assets                1.580000e+08
Retained Earnings                   2.264000e+09
Other Liab                          1.292700e+10
Good Will                           4.091000e+09
Treasury Stock                     -6.331000e+09
Other Assets                        3

In [4]:
info = msft.info
#print(info)
print('Dividend Rate',info['dividendRate'])
print('Annual Dividend Yield',info['trailingAnnualDividendYield'])
print('Forward PE (uses forcasted earnings)',info['forwardPE'])
print('Trailing PE (based on last 12 monthes of earning)',info['trailingPE']) #ideally less than 9
print('Price to Book Value:',info['priceToBook']) #Ideally less than 1.2 #book value is total Assets-total Liabilities
print('Forward Earnings Per Share (uses forcasted earnings):',info['forwardEps'])
print('Trailing Earning Per Share (uses last 12 monthes of earning)',info['trailingEps'])

#print(info)

Dividend Rate 2.24
Annual Dividend Yield 0.009755414
Forward PE (uses forcasted earnings) 28.69612
Trailing PE (based on last 12 monthes of earning) 34.579773
Price to Book Value: 13.14044
Forward Earnings Per Share (uses forcasted earnings): 7.47
Trailing Earning Per Share (uses last 12 monthes of earning) 6.199


## Practice Writing into Pandas Dataset
- writing additional rows [tutorial](https://appdividend.com/2020/01/22/python-pandas-how-to-add-rows-in-dataframe/)

In [5]:
df = pd.DataFrame(columns = ['Ticker', 'P/E','Dividend','Price to Book'])
nr = ['MSFT',10,1,3]

df = df.append(pd.Series(nr,
                         index = df.columns),
               ignore_index=True)
print (df)

  Ticker P/E Dividend Price to Book
0   MSFT  10        1             3


## Use Yahoo Finance to Get Names of All Stocks
- Useful [tutorial](https://algotrading101.com/learn/yahoo-finance-api-guide/) on yfin
- yfin has the dependency of requests_html which is NOT installed by default in Anaconda

## Notes
- When loading all of the others list there are many funds to invest in. I imagine all of these funds will have a PE of either 0 or NA. Since my script will not be able to evaluate funds I am going to need a way to ignore them automatically 

In [3]:
import yahoo_fin.stock_info as si
dow_list = si.tickers_dow()
nasdaq_list = si.tickers_nasdaq()
other_list = si.tickers_other()
sp500_list = si.tickers_sp500()

print('Number Stocks in SP500',len(sp500_list))
print('Number Stocks in Other',len(other_list))
print('Number Stocks in Nasdaq',len(nasdaq_list))
print('Number Stocks in Dow',len(dow_list))

Number Stocks in SP500 505
Number Stocks in Other 5557
Number Stocks in Nasdaq 3870
Number Stocks in Dow 30


## Checking Outputs for a Fund
- Looking for methods to ignore funds
- Testing done on [DOG](https://finance.yahoo.com/quote/DOG?p=DOG&.tsrc=fin-srch)

## Results
- for dog the balance sheet returned an empty array since funds do not necessarily have a balance sheet since they are just investments in various other stocks managed by others
- Using dog.info takes about 540 us to execute. In contrast dog.balance_sheet takes about 2.57 seconds. CORRECTION: Whichever request goes first takes a long time, whatever request is second goes fast. I imagine this has to do with how data connections are made

In [13]:
import time as tm
dog = yf.Ticker("DOG")

ts = tm.time_ns()
dgInfo = dog.info
te = tm.time_ns()
print((te-ts)/np.power(10,9))

ts = tm.time_ns()
dgBal = dog.balance_sheet
te = tm.time_ns()
print((te-ts)/np.power(10,9))
#print(dgInfo)



2.214624
6.5e-05


## Run a Looped Script Over the DOW (30 Stocks)
- I should store all the data of interest in a single dataframe and then loop over that later. That way I will have all of the data so that if I adjust any of my requirements I won't need to redownload everything

## Notes
- The looping through the 
- I can run through everything once and then store the names of all the actual stocks. This will speed up subsequent runs and could also be updated occasionally to stay up to date. 
- I believe that in the yfinance format that if a stock has a negative P/E ratio that a trailing EPS will not be displayed. The same might be true of a negative forward Eps. This statement can be confirmed after further experimentation
- After iterating through multiple stocks it is possible to find ones with negative forward PE. I think this was the last thing I needed to definatively say that Forward PE can be validly used to 
- Trailing EPS can be negative but PE ratio cannot. So first check that forward PE ratio is existing to confirm it is a stock. Next confirm EPS is non-negative using either a call to stk.earnings or the EPS provided by the stock
- Some stocks like A are missing information suchs as Assets and Liabilities I will have to see if this is also an issue in IEXtrade. This is true for a LOT of the stocks in OTHER. This is because they were never supposed to have that information. All of that info is in the balance_sheet call

In [4]:
stkDF = pd.DataFrame(columns = ['Ticker','Forward P/E','Trailing P/E','Trailing Dividend Yield','Price to Book',
                            'Current Assets','Current Liabilities','Total Assets','Total Liabilities','Inventory'])

for stk in other_list:
    stkDat = yf.Ticker(stk)
    if '$' not in stk and '.' not in stk:
        stkInfo = stkDat.info
        #stkEarn = stkDat.quarterly_earnings
        #print(stkEarn['Earnings'])
        if 'forwardPE' in stkInfo and stkInfo['forwardPE'] != None:
            #At this point I know I am dealing with an actual stock and not some kind other investment
            print(stk,'Forward:',stkInfo['forwardPE'])
            forPE = stkInfo['forwardPE']
            if  stkInfo['trailingEps'] > 0:
                #print(stk,'Traling:',stkInfo['trailingPE'])
                trlPE = stkInfo['trailingPE'] #Reads the trailing PE ratio
            else:
                #print(stk,'Trailing:','Negative')
                trlPE = -1
            divRate = stkInfo['trailingAnnualDividendYield'] #Reads the trailing dividend info
            p2b = stkInfo['priceToBook'] 
            stkBal = stkDat.balance_sheet
            recBal = stkBal[stkBal.columns[0]]
            curAsst = recBal['Total Current Assets'] if 'Total Current Assets' in recBal else 'None'
            curLiab = recBal['Total Current Liabilities'] if 'Total Current Liabilities' in recBal else 'None'
            totAss = recBal['Total Assets'] if 'Total Assets' in recBal else 'None'
            totLiab = recBal['Total Liab'] if 'Total Liab' in recBal else 'None'
            inv = recBal['Inventory'] if 'Inventory' in recBal else 'None'
            nr = [stk,forPE,trlPE,divRate,p2b,curAsst,curLiab,totAss,totLiab,inv]
            stkDF = stkDF.append(pd.Series(nr,
                         index = stkDF.columns),
               ignore_index=True)  
            

A Forward: 28.263287
AA Forward: 54.162792
AAN Forward: 8.835897
AAP Forward: 16.082106
AAT Forward: 41.083332


KeyboardInterrupt: 

In [6]:
print(stkDF)
#print(stkDF['Ticker','Inventory'])
print(other_list)

  Ticker  Forward P/E  Trailing P/E  Trailing Dividend Yield  Price to Book  \
0      A    28.263287     50.873917                 0.006294       7.347567   
1     AA    54.162792     -1.000000                 0.015391       1.275465   
2    AAN     8.835897     -1.000000                 0.008533       0.710340   
3    AAP    16.082106     22.241957                 0.005271       2.760452   
4    AAT    41.083332     47.027027                 0.034930       1.392065   

   Current Assets  Current Liabilities  Total Assets  Total Liabilities  \
0    3.415000e+09         1.467000e+09  9.627000e+09       4.754000e+09   
1    3.530000e+09         2.563000e+09  1.463100e+10       8.745000e+09   
2    9.388760e+08         4.073390e+08  1.940331e+09       1.102550e+09   
3    5.695543e+09         4.477702e+09  1.124852e+10       7.699444e+09   
4    1.812970e+08         6.599800e+07  2.790333e+09       1.496661e+09   

     Inventory  
0      7.2e+08  
1    1.644e+09  
2  7.81598e+08  
3  4.4

In [100]:
stkDat = yf.Ticker('AAP')
stkInfo = stkDat.info
stkFin = stkDat.financials
print(stkFin)
stkBal = stkDat.balance_sheet
print(stkBal)

                                         2019-12-28   2018-12-29   2017-12-30  \
Research Development                           None         None         None   
Effect Of Accounting Charges                   None         None         None   
Income Before Tax                       6.37746e+08  5.55264e+08  5.20259e+08   
Minority Interest                              None         None         None   
Net Income                              4.86896e+08  4.23847e+08  4.75505e+08   
Selling General Administrative          3.47381e+09  3.46259e+09  3.50154e+09   
Gross Profit                            4.25809e+09  4.22615e+09  4.08505e+09   
Ebit                                    7.56783e+08  7.25542e+08  5.83512e+08   
Operating Income                        7.56783e+08  7.25542e+08  5.83512e+08   
Other Operating Expenses                       None         None         None   
Interest Expense                        -3.9898e+07  -5.6588e+07  -5.8801e+07   
Extraordinary Items         

## Getting Previous Earnings Data Notes
- Past Earnings Data can be viewed quarterly by calling msft.quarterly_earnings
- Past Earnings Data yearly for the last four years can be obtained by calling msft.earnings

In [29]:
save = yf.Ticker("SAVE")

quartDat = save.quarterly_earnings
quartEarn = quartDat['Earnings']
print(quartEarn)
print(quartEarn[0])

Dat = save.earnings
print(Dat)


Quarter
4Q2019     81214000
1Q2020    -27828000
2Q2020   -144428000
3Q2020    -99140000
Name: Earnings, dtype: int64
81214000
         Revenue   Earnings
Year                       
2016  2320021000  263483000
2017  2643552000  415522000
2018  3323034000  155749000
2019  3830536000  335255000


## Getting Historical Stock Data
- I need to get earning for multiple recent quarters
- Requesting historical information with the hist function only returns the Price and dividend
- I can get the required data just by using a call to earnings

In [35]:
msft = yf.Ticker("MSFT")
hist = msft.history(period="max")

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
1986-03-13,0.06,0.06,0.06,0.06,1031788800,0.0,0.0
1986-03-14,0.06,0.07,0.06,0.06,308160000,0.0,0.0
1986-03-17,0.06,0.07,0.06,0.07,133171200,0.0,0.0
1986-03-18,0.07,0.07,0.06,0.06,67766400,0.0,0.0
1986-03-19,0.06,0.06,0.06,0.06,47894400,0.0,0.0
...,...,...,...,...,...,...,...
2020-11-30,214.10,214.76,210.84,214.07,33064800,0.0,0.0
2020-12-01,214.51,217.32,213.35,216.21,30931300,0.0,0.0
2020-12-02,214.88,215.47,212.80,215.37,23724500,0.0,0.0
